In [1]:
from cil.io import NEXUSDataReader, NEXUSDataWriter
from cil.optimisation.algorithms import SPDHG
from cil.plugins.astra.operators import ProjectionOperator
from cil.optimisation.functions import IndicatorBox, L2NormSquared, MixedL21Norm, BlockFunction
from cil.optimisation.operators import GradientOperator, BlockOperator
from cil.framework import BlockDataContainer
from cil.processors import Slicer
from cil.optimisation.operators import GradientOperator

import pickle

In [2]:
# Load data after the RingRemover processor
name = "data_after_ring_remover_318_398.nxs"
read_data = NEXUSDataReader(file_name = "HyperspectralData/" + name)
data = read_data.load_data()
print("Sinogram shape is {}".format(data.shape))

Sinogram shape is (80, 80, 120, 400)


### For this notebook, we select only 5 energy channels and 5 vertical slices. 

### To reproduce the results in the paper, comment the line below. The reconstruction with 10 subsets and 500 iterations takes about 6 hours.

### In the following, we would like to reproduce (mainly) Figure 11 and compare it with the PDHG reconstruction from the [PDHG_SpatioSpectralTV.ipynb](PDHG_SpatioSpectralTV.ipynb).

**Note:** We run for only 1000 iterations, i.e., 50 epochs.

In [3]:
# comment to reproduce the results in the paper
data = Slicer(roi={'channel': (37,42),'vertical': (17,22)})(data)
print("Sinogram shape is {}".format(data.shape))

# Extract geometry and angle information from data
ag = data.geometry
ig = ag.get_ImageGeometry()
angles = ag.angles

Sinogram shape is (5, 5, 120, 400)


In [4]:
# Setup SPDHG parameters
sample = "stride"

# Choose number of subsets
subsets = 10
size_of_subsets = int(len(angles)/subsets)

if sample =="uniform":
    list_data = [Slicer(roi={'angle':(i,len(angles),i+size_of_subsets)})(data) for i in range(0, len(angles), size_of_subsets)]
elif sample =="stride":  
    list_data = [Slicer(roi={'angle':(i,len(angles),subsets)})(data) for i in range(subsets)]
            
list_geometries = [ag.geometry for ag in list_data ]

# GradientOperator
Grad = GradientOperator(ig, correlation="SpaceChannels")

# For every geometry in list_geometries, define operators A_i
A_i = []
A_i = [ProjectionOperator(ig, ageom) for ageom in list_geometries]
A_i.append(Grad)

# Wrap the projection operators A_i and GradientOperator to the BlockOperator K
K = BlockOperator(*A_i)

b = BlockDataContainer(*list_data)
    
# List of probabilities
prob = [1/(2*subsets)]*(subsets) + [1/2]   

# Regularisation parameter
alpha = 0.002 
   
# List of BlockFunctions
fsubsets = [0.5*L2NormSquared(b = b[i]) for i in range(subsets)]
f = BlockFunction(*fsubsets, alpha * MixedL21Norm())

# Positivity constraint of G
g = IndicatorBox(lower=0)

Initialised GradientOperator with C backend running with  16  threads


### We use callback_save to save the reconstruction every 100 iterations. 

**Note:** Under the explicit (SPDHG) setup, one epoch corresponds to 20 iterations. Therefore, 5 epochs
correspond to 100 iterations. 

In [5]:
def callback_save(iteration, objective_value, solution):
    """Callback function to save images"""
    
    list_iter = [z for z in range(0,1001,100)]        

    if iteration in list_iter:
        name = "spdhg_spatiospectral_tv_reconstruction_iter_{}.nxs".format(iteration)
        writer = NEXUSDataWriter(file_name = "HyperspectralData/" + name,
                                 data = solution)     
        writer.write() 
        
# Run SPDHG algorithm
spdhg = SPDHG(f = f, g = g, operator = K, 
              max_iteration = 1001,
              update_objective_interval = 1, prob = prob )
spdhg.run(verbose=0, callback=callback_save) 

spdhg_info = {}
spdhg_info['timing'] = spdhg.timing
with open('HyperspectralData/objectives_spdhg_spatiospectral_tv.pkl','wb') as f:
    pickle.dump(spdhg_info, f)    

SPDHG setting up
SPDHG configured


/home/vaggelis/miniconda3/envs/cil21_tigre_astra/lib/python3.8/site-packages/cil/framework/framework.py:2185: RuntimeWarning: invalid value encountered in true_divide
  pwop(self.as_array(), x2.as_array(), *args, **kwargs )
